## Imports

In [1]:
import time
import json
import numpy as np
import pandas as pd
from pymatgen.core.periodic_table import Species
from pymatgen.core.structure import Structure

## Read in JSON for Testing

In [2]:
with open('Materials_Project_NaNbO3.json') as json_file:
    nanbo3_dct = json.load(json_file)
for key in list(nanbo3_dct.keys()):
    nanbo3_dct[key]['structures'] = [Structure.from_dict(s) for s in nanbo3_dct[key]['structures']]

# Benchmark gii_calculator.py

## Compute Matminer GII - Benchmarking

In [3]:
from matminer.featurizers.structure.bonding import GlobalInstabilityIndex
# matminer GlobalInstabilityIndex uses bvparm16.cif, which is the default for GIICalculator

In [4]:
benchmark_structure = nanbo3_dct['NaNbO3']['structures'][1]

In [5]:
start_time = time.time()
mm_gii_calc = GlobalInstabilityIndex(r_cut=4)
mm_gii = mm_gii_calc.featurize(benchmark_structure)[0]
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

/Users/RyanMorelock/anaconda3/lib/python3.7/site-packages/pandas/io/parsers/readers.py:586: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  return _read(filepath_or_buffer, kwds)


Time taken: --- 0.9626851081848145 seconds ---


In [6]:
mm_gii

0.22741774664473702

## Compute GIICalculator GII

In [7]:
from gii_calculator import GIICalculator

In [8]:
start_time = time.time()
#gii_calc = GIICalculator(method='CrystalNN')
gii_calc = GIICalculator(method='Cutoff', cutoff=4)
gii = gii_calc.GII(benchmark_structure)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.19856786727905273 seconds ---


In [9]:
gii

0.22741774664473707

In [10]:
gii_calc.params_dict

{'Cation': [Species Na+, Species Nb5+],
 'Anion': [Species O2-, Species O2-],
 'R0': [1.803, 1.911],
 'B': [0.37, 0.37]}

# Benchmark site_optimization.py

## Perform the Site Optimization

In [11]:
from site_optimization import SiteClusterOptimization

In [18]:
sco = SiteClusterOptimization()

In [19]:
start_time = time.time()
final_structure = sco.minimize_cluster_di_squared(benchmark_structure, site_ind=16) # pass the index of site to be optimized
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 3.984825849533081 seconds ---


In [20]:
start_time = time.time()
start_gii = gii_calc.GII(benchmark_structure)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.14157724380493164 seconds ---


In [21]:
start_gii

0.22741774664473707

In [22]:
start_time = time.time()
final_gii = gii_calc.GII(final_structure, use_sym=False)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.3294098377227783 seconds ---


In [23]:
final_gii

0.2222068229123708

## Perform the Structure Optimization 

In [12]:
from site_optimization import GIIMinimizer

In [25]:
gii_minimizer = GIIMinimizer(benchmark_structure, convergence_tolerance=0.01)
# All sites optimized until optimization of said site changes GII by < convergence_tolerance

In [26]:
optimized_structure = gii_minimizer.gii_minimization(opt_method='max')

Step 1 complete; 0.22741774664473727 --> 0.21685468242809414
Step 2 complete; 0.21685468242809414 --> 0.21253671761744256
Step 3 complete; 0.21253671761744256 --> 0.20319583095480914
Step 4 complete; 0.20319583095480914 --> 0.19929305787714166
Step 5 complete; 0.19929305787714166 --> 0.17406125117144214
Step 6 complete; 0.17406125117144214 --> 0.1541425987472521
Step 7 complete; 0.1541425987472521 --> 0.14735253314386446
Step 8 complete; 0.14735253314386446 --> 0.12605574185146776
Step 9 complete; 0.12605574185146776 --> 0.1172540398089425
Step 10 complete; 0.1172540398089425 --> 0.11532954906197346
Step 11 complete; 0.11532954906197346 --> 0.11098630269185894
Step 12 complete; 0.11098630269185894 --> 0.10514996486333666
Step 13 complete; 0.10514996486333666 --> 0.10514996486333666
Step 14 complete; 0.10514996486333666 --> 0.10514996486333666
Step 15 complete; 0.10514996486333666 --> 0.10512042010554813
Step 16 complete; 0.10512042010554813 --> 0.1051185729262532
Step 17 complete; 0.10

In [27]:
# Get the change in GII following optimization for each site in the structure, show that it is < convergence_tolerance
for i in range(len(gii_minimizer.diffs)):
    print('Specie: %s, delGII of optimization step: %s' % (optimized_structure[i].specie, gii_minimizer.diffs[i]))

Specie: Na+, delGII of optimization step: 0.006511574041070112
Specie: Na+, delGII of optimization step: 0.007380391101802525
Specie: Na+, delGII of optimization step: 0.0018892742339008256
Specie: Na+, delGII of optimization step: 0.00614137470116205
Specie: Nb5+, delGII of optimization step: 2.9544757788532716e-05
Specie: Nb5+, delGII of optimization step: 0.0
Specie: Nb5+, delGII of optimization step: 1.847179294925394e-06
Specie: Nb5+, delGII of optimization step: 0.0
Specie: O2-, delGII of optimization step: 0.008801702042525256
Specie: O2-, delGII of optimization step: 0.001924490746969043
Specie: O2-, delGII of optimization step: 0.004343246370114517
Specie: O2-, delGII of optimization step: 0.0058363378285222756
Specie: O2-, delGII of optimization step: 0.006790065603387657
Specie: O2-, delGII of optimization step: 2.7755575615628914e-17
Specie: O2-, delGII of optimization step: 0.002665507676445314
Specie: O2-, delGII of optimization step: 0.0010834205073378123
Specie: O2-, de

# Benchmark parameterization.py

In [17]:
from parameterization import GeneralBVParamOptimization
from parameterization import GeneralBVParamOptimizationOuterLoop
from pymatgen.core.periodic_table import Specie

In [26]:
### Only consider structures with 20 atoms or fewer to speed up parameterization test
cmpd = 'NaNbO3'
ss = nanbo3_dct[cmpd]['structures']
short_inds = [i for i in range(len(ss)) if len(ss[i]) <= 20]

short_structures = [ss[i] for i in short_inds]
short_energies = [nanbo3_dct['NaNbO3']['energies'][i] for i in short_inds]
short_dct = {cmpd: {'structures': short_structures, 'energies': short_energies}}

cation = Specie('Nb', 5)
anion = Specie('O', -2)
cations_anions = [(Specie('Nb', 5), Specie('O', -2)), (Specie('Na', 1), Specie('O', -2))]
options = {'gtol': 0.01, 'xtol': 0.01, 'barrier_tol': 0.01, 'disp': True, 'verbose': 0} # Changed from default

## Composition-Specific Parameterization

## Parameterization that minimizes site discrepancy factor RMSD

### Single Parameter Optimization

In [13]:
single_parameterization_rmsd = GeneralBVParamOptimization([short_structures], [short_energies], 
                                                          gii_calc.params_dict, cation, anion, 
                                                          obj_func='di2_rmsd', options=options)

In [18]:
rmsd_optimized_param_dict = single_parameterization_rmsd.param_optimizer()

0.43217634981643743
0.43217600192001077
63.21939147166732
63.219399469371446
0.41408252059348816
0.4140829450589659
0.6421104546455951
0.6421101238157407
0.20743101112192222
0.20743065676704303
0.3224547408298752
0.3224551532085578
0.0742907518863612
0.07429094325818793
0.20907176225894358
0.20907140775478625
0.0802351223950141
0.080234895341338
`xtol` termination condition is satisfied.
Number of iterations: 9, function evaluations: 18, CG iterations: 8, optimality: 6.60e+00, constraint violation: 0.00e+00, execution time: 1.9e+01 s.


In [19]:
rmsd_optimized_param_dict, gii_calc.params_dict

({'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.947],
  'B': [0.37, 0.37]},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]})

### Multiple Parameter Optimization

In [30]:
rmsd_pol = GeneralBVParamOptimizationOuterLoop(short_dct, cations_anions, gii_calc.params_dict)
rmsd_pol.parameter_optimization(obj_func='di2_rmsd', init_steps=1)

(Species Nb5+, Species O2-) 1
1.911 0.37
0.43217634981643743
0.43217600192001077
63.21939147166732
63.219399469371446
0.41408252059348816
0.4140829450589659
0.6421104546455951
0.6421101238157407
0.20743101112192222
0.20743065676704303
0.3224547408298752
0.3224551532085578
0.0742907518863612
0.07429094325818793
0.20907176225894358
0.20907140775478625
0.0802351223950141
0.080234895341338
`xtol` termination condition is satisfied.
Number of iterations: 9, function evaluations: 18, CG iterations: 8, optimality: 6.60e+00, constraint violation: 0.00e+00, execution time: 1.8e+01 s.
1.947 0.37
(Species Na+, Species O2-) 1
1.803 0.37
0.11400219061832283
0.11400217674652065
2.929394163023829
2.9293945290047945
0.11964710051488511
0.11964712771839521
0.11232720162549459
0.11232720889891686
0.11177575186306263
0.11177575169816566
0.11177546249795854
0.11177546249600917
`gtol` termination condition is satisfied.
Number of iterations: 6, function evaluations: 12, CG iterations: 5, optimality: 7.22e-

In [29]:
rmsd_pol.starting_params, rmsd_pol.updated_params

({'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.811, 1.947],
  'B': [0.37, 0.37]})

## Parameterization considering GII-DFT Energetics Relationship

### Single Parameter Optimization

In [21]:
single_parameterization_gii = GeneralBVParamOptimization([short_structures], [short_energies], 
                                                         gii_calc.params_dict, cation, anion, options=options)

In [22]:
gii_optimized_param_dict = single_parameterization_gii.param_optimizer()

-0.31195916177186295
-0.3119589033101902
0.8266033357753323
0.8266033403357762
0.8535480718045818
0.8535480312807145
-0.39097757443995895
-0.39097744673730667
0.8219541046955192
0.8219541054515607
0.8597353420839104
0.8597352955922237
-0.006678240659796331
-0.006677508095843554
0.8637758922263825
0.8637758420623807
0.8898997584077339
0.8898996904176671
0.7128752962143169
0.7128769171081397
0.9013424633844567
0.9013423911335877
`gtol` termination condition is satisfied.
Number of iterations: 11, function evaluations: 22, CG iterations: 10, optimality: 4.62e-03, constraint violation: 0.00e+00, execution time: 5.8e+01 s.


In [25]:
gii_optimized_param_dict, gii_calc.params_dict

({'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.961],
  'B': [0.37, 0.37]},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]})

### Multiple Parameter Optimization

In [31]:
gii_pol = GeneralBVParamOptimizationOuterLoop(short_dct, cations_anions, gii_calc.params_dict)

In [32]:
gii_pol.parameter_optimization(init_steps=1, max_steps=3)

(Species Nb5+, Species O2-) 1
1.911 0.37
-0.31195916177186295
-0.3119589033101902
0.8266033357753323
0.8266033403357762
0.8535480718045818
0.8535480312807145
-0.39097757443995895
-0.39097744673730667
0.8219541046955192
0.8219541054515607
0.8597353420839104
0.8597352955922237
-0.006678240659796331
-0.006677508095843554
0.8637758922263825
0.8637758420623807
0.8898997584077339
0.8898996904176671
0.7128752962143169
0.7128769171081397
0.9013424633844567
0.9013423911335877
0.9075435169588297
0.9075434781790628
0.9080191392277475
0.9080191566813389
0.9080651549873842
0.9080651646458515
0.908076014635621
0.9080760214093756
`gtol` termination condition is satisfied.
Number of iterations: 17, function evaluations: 30, CG iterations: 14, optimality: 8.65e-04, constraint violation: 0.00e+00, execution time: 7.9e+01 s.
1.957 0.37
(Species Na+, Species O2-) 1
1.803 0.37
0.9080332128113098
0.9080331561312214
0.8647245005521961
0.8647245075482657
0.928128455842581
0.9281284732688118
0.9102386228649307

In [33]:
gii_pol.starting_params, gii_pol.updated_params

({'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.803, 1.911],
  'B': [0.37, 0.37]},
 {'Cation': [Species Na+, Species Nb5+],
  'Anion': [Species O2-, Species O2-],
  'R0': [1.784, 1.958],
  'B': [0.37, 0.37]})